In [2]:
# imports
import pandas as pd
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
# load data
df = pd.read_csv("top_rated_wines.csv")
df = df[df["variety"].notna()]  # remove any NaN values as it blows up serialization
data = df.to_dict("records")

In [4]:
# create embeddings
encoder = SentenceTransformer("all-MiniLM-L6-v2")

In [5]:
# create the vector database client
vdb = QdrantClient(":memory:")  # create in-memory Qdrant instance

In [6]:
# create the collection
vdb.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

/var/folders/1t/stmkpc7x67x4qstrsg949vdc0000gn/T/ipykernel_47267/1242576402.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  vdb.recreate_collection(


True

In [7]:
# vectorize
# note that for Coursera we use an older way of Qdrant doing the uploads using Records instead of Points
vdb.upload_records(
    collection_name="top_wines",
    records=[
        models.Record(id=idx, vector=encoder.encode(doc["notes"]).tolist(), payload=doc)
        for idx, doc in enumerate(data)  # data is the variable holding all the wines
    ],
)

/var/folders/1t/stmkpc7x67x4qstrsg949vdc0000gn/T/ipykernel_47267/3002580822.py:3: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  vdb.upload_records(


In [18]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [31]:
# search locally
hits = vdb.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3,
)
for hit in hits:
    print(hit.payload['name'], hit.payload['region'], "score:", hit.score)

Catena Zapata Argentino Vineyard Malbec 2004 Argentina score: 0.6377782384717134
Bodega Colome Altura Maxima Malbec 2012 Salta, Argentina score: 0.6179680846507815
Catena Zapata Adrianna Vineyard Malbec 2004 Argentina score: 0.611757557482429


/var/folders/1t/stmkpc7x67x4qstrsg949vdc0000gn/T/ipykernel_47267/1174297874.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = vdb.search(


In [11]:
# naive check if embeddings are stored
vdb.scroll(
    collection_name="top_wines",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(key="variety", match=models.MatchValue(value="Red Wine")),
        ]
    ),
    limit=3,
    with_payload=True,
    with_vectors=False,
)

([Record(id=0, payload={'name': '3 Rings Reserve Shiraz 2004', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Vintage Comments : Classic Barossa vintage conditions. An average wet Spring followed by extreme heat in early February. Occasional rainfall events kept the vines in good balance up to harvest in late March 2004. Very good quality coupled with good average yields. More than 30 months in wood followed by six months tank maturation of the blend prior to bottling, July 2007. '}, vector=None, shard_key=None, order_value=None),
  Record(id=1, payload={'name': 'Abreu Vineyards Cappella 2007', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Cappella is a proprietary blend of two clones of Cabernet Sauvignon with Cabernet Franc, Petit Verdot and Merlot. The gravelly soil at Cappella produces fruit that is very elegant in structure. The resulting wine exhibits beautiful purity of fru

In [20]:
# store search results to be used by llm
search_results = [hit.payload for hit in hits]

In [ ]:
# LLM interaction with openai lib
from openai import OpenAI
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="sk-no-key-required",
)
completion = client.chat.completions.create(
    model="llama3.1",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": str(search_results)},
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content=" \nBased on your request, I highly recommend the **Bodega Colome Altura Maxima Malbec 2012** from Salta, Argentina. This wine has been rated 96 points and is a beautiful example of Argentine Malbec. It's known for its rich flavors of plum, blackberry, and spices with smooth tannins. The notes also show that the winemaker believes it's an embodiment of two extremes - traditional grape variety from his French origins made from the vineyard that challenges all convention in modern viticulture.\n\nYou can order this wine through various online retailers or ask your local wine shop to carry it for you. If you'd like more recommendations, feel free to provide me with a budget and personal taste preferences!", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [30]:
# LLM interaction with langchain lib
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

llm = OllamaLLM(model="llama3.1")
template = """
You are chatbot, a wine specialist. 
Your top priority is to help guide users into selecting amazing wine and guide them with their requests.
Answer the question below using the conversation history and the search results content provided.
Here is the conversation history: {context}
Here are the search results: {search_results}
Question: {question}
Answer:
"""
context = ""
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
response = chain.invoke({"context": context, "search_results": str(search_results), "question": user_prompt})
print(response)

Based on your request for a fantastic Malbec wine from Argentina, I would highly recommend the **Catena Zapata Adrianna Vineyard Malbec 2004**.

This wine has received a stellar rating of 97.0 and is described as "opulent, full-flavored, yet remarkably light on its feet". The notes mention aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur, indicating a complex and pleasing flavor profile.

It's worth noting that this wine has been aged for 17 months in new French oak, which contributes to its smooth and elegant taste. While it may be tempting to open it now, the rating suggests that it will continue to evolve and improve with time, making it a great investment for those who can wait a few years.

If you're looking for an alternative, I'd also suggest considering the **Catena Zapata Argentino Vineyard Malbec 2004**, which has a similar high rating of 98.0 and is described as "remarkably fragrant and complex aromatically".

Let me know if you have any other pr

In [ ]:
# LLM interaction with ollama lib
from ollama import Client
llm = Client(host="http://localhost:11434/v1")
response = llm.chat(model="llama3.2", messages=[
    {"role": "system", "content": "You are Alfred, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests. You are an expert about wine knowledge and especially good at picking best wines for your users."},
    {"role": "user", "content": "I would like a wine that goes well with steak."},
])
print(response.message.content)

ResponseError: 404 page not found (status code: 404)